In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import DeepFMKit.core as dfm
from scipy.special import jv
from scipy.linalg import inv
from scipy.stats import norm

def calculate_jacobian(ndata, param):
    """
    Calculates the Jacobian matrix (J) of the DFMI model.
    (This function remains unchanged from the previous version)
    """
    a, m, phi, psi = param
    J = np.zeros((2 * ndata, 4))
    j = np.arange(1, ndata + 1)
    
    phase_term = np.cos(phi + j * np.pi / 2.0)
    cos_jpsi = np.cos(j * psi)
    sin_jpsi = np.sin(j * psi)
    
    bessel_j = jv(j, m)
    bessel_deriv = 0.5 * (jv(j - 1, m) - jv(j + 1, m))
    
    common_term = a * phase_term * bessel_j
    model_q = common_term * cos_jpsi
    model_i = -common_term * sin_jpsi
    
    if a != 0:
        J[:ndata, 0] = model_q / a
        J[ndata:, 0] = model_i / a

    common_deriv_term_m = a * phase_term * bessel_deriv
    J[:ndata, 1] = common_deriv_term_m * cos_jpsi
    J[ndata:, 1] = -common_deriv_term_m * sin_jpsi

    phase_deriv_term = np.cos(phi + j * np.pi / 2.0 + np.pi / 2.0)
    common_deriv_term_phi = a * phase_deriv_term * bessel_j
    J[:ndata, 2] = common_deriv_term_phi * cos_jpsi
    J[ndata:, 2] = -common_deriv_term_phi * sin_jpsi

    J[:ndata, 3] = common_term * -sin_jpsi * j
    J[ndata:, 3] = -common_term * cos_jpsi * j
    
    return J

# In analyze_precision.py

def calculate_m_precision(m_range, ndata, snr_db, buffer_size):
    """
    Core calculation function for statistical uncertainty of 'm'.
    
    This version correctly accounts for the averaging gain from using
    a buffer of a given size.

    Parameters
    ----------
    m_range : array_like
        The range of modulation depths 'm' to analyze.
    ndata : int
        Number of harmonics to use in the fit.
    snr_db : float
        Signal-to-Noise Ratio in dB, defined for the time-domain signal.
    buffer_size : int
        The number of samples (R) in the buffer being averaged. This
        is the source of the processing gain.

    Returns
    -------
    numpy.ndarray
        An array of the statistical uncertainty (delta_m) for each m in m_range.
    """
    param_fixed = np.array([1.0, 0, np.pi/4, 0.0]) # a, m, phi, psi
    
    # SNR is a power ratio, so we use 10*log10.
    snr_linear_power = 10**(snr_db / 10.0)
    
    # Assume signal amplitude a=1. Signal power is a^2/2 = 0.5.
    signal_power = 0.5
    noise_power_time_domain = signal_power / snr_linear_power
    
    # The variance on the I/Q measurement is the time-domain noise variance
    # divided by the buffer size (averaging gain). A factor of 2 is also
    # included for the properties of quadrature demodulation.
    noise_variance_iq = noise_power_time_domain / (2 * buffer_size)
    
    delta_m_list = []
    for m_true in m_range:
        param_fixed[1] = m_true
        J = calculate_jacobian(ndata, param_fixed)
        JTJ = J.T @ J
        try:
            # The CRLB for the parameters
            covariance_matrix = noise_variance_iq * inv(JTJ)
            delta_m = np.sqrt(covariance_matrix[1, 1])
            delta_m_list.append(delta_m)
        except np.linalg.LinAlgError:
            delta_m_list.append(np.inf)
            
    return np.array(delta_m_list)

In [ ]:
def validate_fitter_efficiency(m_true=15.5, ndata=15, snr_db=70.0, n_trials=500):
    """
    Performs a Monte Carlo simulation to test the NLS fitter's performance
    against the theoretical Cramér-Rao Lower Bound (CRLB).

    This function verifies that the fitter is statistically "efficient" by
    comparing its measured precision from repeated noisy simulations against
    the theoretical best-case precision predicted by the CRLB. The CRLB
    calculation is corrected to account for the processing gain from
    averaging over the sample buffer.

    Parameters
    ----------
    m_true : float, optional
        The ground-truth modulation depth to test.
    ndata : int, optional
        The number of harmonics to use in the fit.
    snr_db : float, optional
        The target Signal-to-Noise Ratio (in dB) for the simulation. The SNR
        is defined as the ratio of the AC signal power to the time-domain
        white noise power.
    n_trials : int, optional
        The number of Monte Carlo trials to run. A higher number gives a
        more reliable estimate of the measured precision.
    """
    # --- 1. Setup & Theoretical Calculation ---
    print("="*60)
    print("Fitter Efficiency Validation: Comparing Measured vs. Theoretical Precision")
    print(f"Parameters: m = {m_true}, ndata = {ndata}, SNR = {snr_db} dB")
    print(f"Number of Monte Carlo trials: {n_trials}")
    print("="*60)

    # Define all simulation parameters in one place for clarity
    sim_params = {
        'm': m_true,
        'amp': 1.0,
        'f_mod': 1000,
        'f_samp': 200000,
        'fit_n': 20
    }

    # Calculate the derived buffer size (R) and duration
    buffer_size = int(sim_params['fit_n'] * (sim_params['f_samp'] / sim_params['f_mod']))
    n_seconds_per_buffer = sim_params['fit_n'] / sim_params['f_mod']

    # --- 2. Calculate the Theoretical CRLB ---
    # This call now includes the buffer_size, which is the crucial correction.
    print("\nCalculating theoretical precision (CRLB)...")
    delta_m_crlb = calculate_m_precision(np.array([m_true]), ndata, snr_db, buffer_size)[0]
    print(f"Theoretical Precision (CRLB): δm = {delta_m_crlb:.4e}")

    # --- 3. Initialize Framework and Run Monte Carlo Simulation ---
    dff = dfm.DeepFitFramework()
    label = "efficiency_test"
    dff.new_sim(label)
    
    # Configure the simulation object from our parameter dictionary
    sim_config = dff.sims[label]
    for key, value in sim_params.items():
        setattr(sim_config, key, value)
    
    m_estimates = []
    # The initial guess for the fitter should be slightly off to ensure it does work.
    initial_guess = np.array([sim_config.amp, m_true * 0.95, 0.1, 0.1])
    
    print(f"\nRunning {n_trials} Monte Carlo simulations...")
    for i in tqdm(range(n_trials)):
        # Use the simple SNR-based simulation method
        dff.simulate(
            label,
            n_seconds=n_seconds_per_buffer,
            snr_db=snr_db,
            trial_num=i  # Ensures each noise realization is unique
        )
        
        # Run the fit on the newly generated data buffer
        fit_result = dff._fit_single_buffer(label, 0, buffer_size, ndata, initial_guess)
        m_estimates.append(fit_result['m'])

    # --- 4. Analyze and Print the Results ---
    m_estimates = np.array(m_estimates)
    delta_m_measured = np.std(m_estimates)
    mean_m_measured = np.mean(m_estimates)
    bias = mean_m_measured - m_true
    
    # Estimator efficiency is the ratio of theoretical best variance to actual variance
    efficiency = (delta_m_crlb**2 / delta_m_measured**2) * 100 if delta_m_measured > 0 else 0

    print("\n--- Results ---")
    print(f"Measured Mean of estimates:  <m> = {mean_m_measured:.6f}")
    print(f"Estimator Bias (<m> - m_true):   = {bias:.4e}")
    print(f"Measured Precision (Std Dev): δm = {delta_m_measured:.4e}")
    print(f"Estimator Efficiency (CRLB² / Measured²): {efficiency:.1f}%")

    # --- 5. Plot the Distribution of Estimates for Visual Confirmation ---
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot the histogram of results
    count, bins, _ = ax.hist(m_estimates, bins=50, density=True, 
                             label=f'Histogram of $\hat{{m}}$ estimates\n($N_{{trials}}={n_trials}$)', 
                             alpha=0.6, color='tab:blue')

    # Overlay a fitted Gaussian curve
    mu, std = norm.fit(m_estimates)
    p = norm.pdf(bins, mu, std)
    ax.plot(bins, p, 'k--', linewidth=2, label='Fitted Normal Distribution')

    # Add vertical lines for key values
    ax.axvline(m_true, color='red', linestyle='-', linewidth=2, label=f'True m = {m_true:.4f}')
    ax.axvline(mean_m_measured, color='black', linestyle='--', linewidth=2, label=f'Measured Mean = {mean_m_measured:.4f}')

    title_text = f"Estimator Performance vs. CRLB (m={m_true}, ndata={ndata}, SNR={snr_db}dB)"
    ax.set_title(title_text, fontsize=16)
    ax.set_xlabel('Estimated Modulation Depth ($\hat{m}$)', fontsize=14)
    ax.set_ylabel('Probability Density', fontsize=14)
    
    # Add a text box with the key results for easy reading on the plot
    results_str = (f"Bias = {bias:.2e}\n"
                   f"$\delta m_{{measured}}$ = {delta_m_measured:.3e}\n"
                   f"$\delta m_{{CRLB}}$ = {delta_m_crlb:.3e}\n"
                   f"Efficiency = {efficiency:.1f}%")
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, results_str, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=props)

    ax.legend()
    ax.grid(True, linestyle=':')
    fig.tight_layout()
    plt.show()

validate_fitter_efficiency(m_true=4.8, ndata=10, snr_db=0.0, n_trials=500)

In [ ]:
def analyze_bias_variance_tradeoff(m_range=np.linspace(2.0, 10.0, 81), 
                                   ndata=10, 
                                   snr_db=70.0, 
                                   n_trials=500):
    """
    Performs a Monte Carlo simulation across a range of modulation depths (m)
    to analyze the estimator's bias-variance trade-off.

    This function measures both the bias and the variance of the NLS fitter
    and compares its variance to the Cramér-Rao Lower Bound (CRLB) to
    calculate its statistical efficiency. The results reveal regions where the
    estimator becomes biased to achieve a lower variance, resulting in an
    apparent efficiency greater than 100%.

    Parameters
    ----------
    m_range : array_like, optional
        The range of ground-truth modulation depths 'm' to test.
    ndata : int, optional
        The number of harmonics to use in the fit.
    snr_db : float, optional
        The target Signal-to-Noise Ratio (in dB) for the simulations.
    n_trials : int, optional
        The number of Monte Carlo trials to run for each value of 'm'.
    """
    # --- 1. Setup ---
    print("="*60)
    print("Analyzing Estimator Bias-Variance Trade-off")
    print(f"m range: {m_range.min()} to {m_range.max()} ({len(m_range)} points)")
    print(f"ndata={ndata}, SNR={snr_db} dB, n_trials={n_trials}")
    print("="*60)

    # Simulation parameters
    sim_params = { 'amp': 1.0, 'f_mod': 1000, 'f_samp': 200000, 'fit_n': 20 }
    buffer_size = int(sim_params['fit_n'] * (sim_params['f_samp'] / sim_params['f_mod']))
    n_seconds_per_buffer = sim_params['fit_n'] / sim_params['f_mod']

    # --- 2. Initialize Framework and Data Storage ---
    dff = dfm.DeepFitFramework()
    label = "tradeoff_test"
    dff.new_sim(label)
    sim_config = dff.sims[label]
    for key, value in sim_params.items():
        setattr(sim_config, key, value)
    
    bias_list = []
    efficiency_list = []
    
    # --- 3. Loop over the m_range and run Monte Carlo for each point ---
    print("\nRunning Monte Carlo analysis for each m value...")
    for m_true in tqdm(m_range):
        sim_config.m = m_true
        m_estimates = np.zeros(n_trials)
        initial_guess = np.array([sim_config.amp, m_true * 0.95, 0.1, 0.1])

        for i in range(n_trials):
            dff.simulate(label, n_seconds=n_seconds_per_buffer, snr_db=snr_db, trial_num=i)
            fit_result = dff._fit_single_buffer(label, 0, buffer_size, ndata, initial_guess)
            m_estimates[i] = fit_result['m']

        # --- 4. Calculate Bias, Variance, and Efficiency for this m_true ---
        delta_m_crlb = calculate_m_precision(np.array([m_true]), ndata, snr_db, buffer_size)[0]
        
        delta_m_measured = np.std(m_estimates)
        mean_m_measured = np.mean(m_estimates)
        
        bias = mean_m_measured - m_true
        
        # Avoid division by zero if a fit completely fails
        if delta_m_measured > 0:
            efficiency = (delta_m_crlb**2 / delta_m_measured**2) * 100
        else:
            efficiency = 0.0

        bias_list.append(bias)
        efficiency_list.append(efficiency)

    # --- 5. Plot the Final Results ---
    bias_array = np.array(bias_list)
    efficiency_array = np.array(efficiency_list)
    
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    # Plot 1: Efficiency on the left y-axis
    color1 = 'tab:blue'
    ax1.set_xlabel('True Modulation Depth (m)', fontsize=14)
    ax1.set_ylabel('Estimator Efficiency (%)', color=color1, fontsize=14)
    ax1.plot(m_range, efficiency_array, color=color1, linewidth=2, label='Measured Efficiency')
    ax1.axhline(100, color=color1, linestyle='--', label='100% Efficiency (CRLB)')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid(True, which='both', linestyle=':')
    
    # Plot 2: Bias on the right y-axis
    ax2 = ax1.twinx()
    color2 = 'tab:red'
    ax2.set_ylabel('Estimator Bias (m_measured - m_true)', color=color2, fontsize=14)
    # Normalize the bias by the CRLB to make it unitless and comparable
    # A value of 1 means the bias is as large as the expected statistical error.
    normalized_bias = bias_array / calculate_m_precision(m_range, ndata, snr_db, buffer_size)
    ax2.plot(m_range, normalized_bias, color=color2, linestyle='-', marker='.', markersize=4, alpha=0.7, label='Normalized Bias ($\sigma$ units)')
    ax2.axhline(0, color=color2, linestyle='--')
    ax2.tick_params(axis='y', labelcolor=color2)
    
    # Final plot aesthetics
    ax1.set_title('Estimator Efficiency and Bias vs. Modulation Depth', fontsize=16)
    fig.tight_layout()
    
    # Combine legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc='upper right')
    
    plt.show()

# Run the analysis with parameters that highlight the issue
# We choose a range around m=4.8 where the effect was seen
m_test_range = np.linspace(3.0, 20.0, 100) # High resolution in the interesting region
analyze_bias_variance_tradeoff(m_range=m_test_range, ndata=10, snr_db=70.0, n_trials=200)